## Classificação por isolamento de frequência

Neste notebook será realizado um exemplo de classificação, isolando todas as frequências estimuladas no conjunto de dados `AVI SSVEP Single Target`. 

### Passos para a realização da classificação:

1. **Carrega** o arquivo `fif` (`mne.EpochsArray`) dos dados **filtrados**;
2. **Determine o limiar** para isolar cada uma das frequências estimuladas. Por exemplo, a faixa de frequência para o estímulo de 6.5 Hz irá resultar em pontos (`PSD`) que irão variar de 6.3 à 6.7 Hz, caso o limiar seja de 0.2 Hz;
3. **Obter a "energia"** do sinal por meio do cálculo `compute_psd` para cada uma das faixas de frequência que podem ser estimuladas. Por exemplo:
    - Obtenha todas as frequências estimuladas. Ex: 6, 6.5, 7, 7.5, 8.2 e 9.3;
    - Obtenha o valor mínimo e o máximo para cada frequência utilizando limiar. Ex: (5.8, 6.2), (6.3, 6.7), ...
    - Aplique o `compute_psd` para cada tupla (min, max), por meio dos parâmetros `fmin` e `fmax` do mesmo método.
4. Com as listas de pontos isoladas e computadas (`PSD`) para cada amostra, aplique um cálculo de característica adequada. Características manuais interessantes para este exemplo podem ser `max_value`, `average` ou `median`. No fim deste passo iremos obter um **vetor de características**;
5. Por fim, realize a **classificação**, que será um **cálculo de voto** simples (maior valor é provavelmente o a frequência evocada).

A seguir, um exemplo desta classificação com os dados `single target` de `AVI dataset`:

In [1]:
import mne
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

data = mne.read_epochs("./mne_data_beta.fif")

Reading /home/kuak/Documentos/RP/RP_SSVEP/ssvep/src/beta/mne_data_beta.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...    1462.89 ms
        0 CTF compensation matrices available
Not setting metadata
160 matching events found
No baseline correction applied
0 projection items activated


In [6]:
print(data.get_data().shape)
print(data)

(160, 9, 750)
<EpochsFIF |  160 events (all good), 0 – 1.46289 s, baseline off, ~8.3 MB, data loaded,
 '8.0': 4
 '8.2': 4
 '8.4': 4
 '8.6': 4
 '8.799999999999999': 4
 '9.0': 4
 '9.2': 4
 '9.4': 4
 '9.6': 4
 '9.8': 4
 and 30 more events ...>


In [5]:
y = np.load("../../datasets/beta/labels_beta.npy")
# print(y)

y_set = sorted(set(y))
print(y_set)


[8.0, 8.2, 8.4, 8.6, 8.799999999999999, 9.0, 9.2, 9.4, 9.6, 9.8, 10.0, 10.2, 10.4, 10.6, 10.8, 11.0, 11.2, 11.4, 11.6, 11.8, 12.0, 12.2, 12.4, 12.600000000000001, 12.8, 13.0, 13.200000000000001, 13.4, 13.600000000000001, 13.8, 14.0, 14.200000000000001, 14.4, 14.600000000000001, 14.8, 15.0, 15.200000000000001, 15.4, 15.600000000000001, 15.8]


In [51]:
# for i in range(len(data)):
#     # view = mne_data.compute_psd(method='welch', fmin=3, fmax=13)
#     view = data[i].compute_psd(method='multitaper', fmin=3, fmax=13, verbose=False)
#     view.plot(show=False)
#     print()
#     plt.title('Domínio da frequência')
#     plt.axvline(x=float(list(data[i].event_id)[0]), linestyle='--', color='green')
#     plt.show()
# print()

### Beta dataset

Freqs estimuladas: [8.0, 8.2, ..., 15.6, 15.8] - (40 targets) <br/>
Passa-faixa: ~6Hz - ~18Hz <br/>
computed_psd: 7Hz-17Hz (Sugestão)<br/>
computed_psd('multitaper') ->  computed_psd('welch') <br/>

(160, 64, 750) <br/>
160 - 4 trials, 40 targets <br/>
64 - channels <br/>
750 - fs: 250Hz, trial: 3s <br/>

> Fx, Fpx, Tx ... -> não evocam SSVEP <br/>
> 64 channels - Px, P0x e 0x são os melhores <br/>
> ARTIGO: Pz, PO3, PO5, PO4, PO6, POz, O1, Oz and O2

EpochsARray -> drop_channels (drop canais não interessantes) <br/>
faixa de 0.1 Hz <br/>

Verificar se todas possuem a mesma quantidade **(transformar em nparray)**: <br/>
    * 7.9 - 8.1 <br/>
    * 8.1 - 8.3 <br/>
    * ... <br/>
    * 15.7 - 15.9 <br/>


64 eletrodos <br/>
SynAmps2 <br/>
10-20 system <br/>

Obs: para obter a lista de canais (chatgpt), utilizado para criar "info"


In [52]:
# limiar = 0.1

# alvos = [float(item) for item in data.event_id.keys()]
# print(f"Alvos: {alvos}")

# features = list()
# for i in range(len(data)):
#     sample = list()
#     for alvo in alvos:
#         freq_min = alvo - limiar
#         freq_max = alvo + limiar
#         sample.append(data[i].compute_psd(method='welch', fmin=freq_min, fmax=freq_max, verbose=False))

#     features.append(sample)
    
# X = np.array(features)
# print("Formato dos dados calculados:", X.shape)

In [4]:
# limiar = 0.1

alvos = [float(item) for item in data.event_id.keys()]
print(f"Alvos: {alvos}")

features = list()
for i in range(len(data)):
#     data[i].compute_psd(method='welch', fmin=7, fmax=17, verbose=False)
    # features.append(data[i].compute_psd(method='welch', fmin=7, fmax=17, verbose=False))
    # for alvo in alvos:
    #     freq_min = alvo - 0.34
    #     freq_max = alvo + 0.34
        
    print(data[i].get_data(tmin=0.5, tmax=2.5))
    
# X = np.array(features)
# print("Formato dos dados calculados:", X.shape)

Alvos: [8.0, 8.2, 8.4, 8.6, 8.799999999999999, 9.0, 9.2, 9.4, 9.6, 9.8, 10.0, 10.2, 10.4, 10.6, 10.8, 11.0, 11.2, 11.4, 11.6, 11.8, 12.0, 12.2, 12.4, 12.600000000000001, 12.8, 13.0, 13.200000000000001, 13.4, 13.600000000000001, 13.8, 14.0, 14.200000000000001, 14.4, 14.600000000000001, 14.8, 15.0, 15.200000000000001, 15.4, 15.600000000000001, 15.8]
[[[ 0.03158694 -0.0304726   0.12248348 ... -1.09226716 -1.55895352
   -1.94381273]
  [-1.2332679  -0.88015878 -0.34938547 ... -1.28564751 -1.52603328
   -1.69515502]
  [-0.79350901 -0.10130143  0.73048931 ... -1.7761004  -1.75015974
   -1.64948177]
  ...
  [-0.81188613 -0.54242373 -0.13269535 ...  0.5064894  -0.22585154
   -0.90266621]
  [ 0.559268    0.92425209  1.33751655 ... -0.35117644 -1.05666411
   -1.68062675]
  [-1.34136951 -0.74834037 -0.09084266 ... -1.25525391 -1.88938761
   -2.43098021]]]
[[[ 3.37942863  3.78546953  4.08014774 ... -3.96773005 -4.60933113
   -5.08908415]
  [ 3.69661713  4.23169518  4.66070461 ... -2.71538496 -3.200

[[[23.7507534  23.86362648 23.58009529 ... -5.46273661 -5.42761564
   -5.26196623]
  [23.91685677 24.23950768 24.16365242 ... -5.76471472 -5.57594633
   -5.25829792]
  [25.38444901 25.73539352 25.66878128 ... -4.83865643 -4.83820486
   -4.71275473]
  ...
  [ 7.13342905  7.63886309  8.02038002 ... -2.27224278 -2.52991557
   -2.71216416]
  [ 8.96866989  8.86831093  8.60438156 ... -2.24291611 -2.42309642
   -2.52728105]
  [ 9.09988308  8.81970215  8.36766052 ... -2.17712593 -2.30195689
   -2.35831928]]]
[[[-1.60810995 -1.6305095  -1.56766427 ...  3.49619794  4.26068306
    4.9147644 ]
  [ 0.2675218   0.06888746 -0.08256385 ...  3.2784133   3.88871408
    4.37862158]
  [ 2.02392697  1.61846459  1.20462489 ...  2.83355188  3.20174527
    3.48372173]
  ...
  [ 2.75979567  3.01752353  3.21391368 ...  3.00724459  3.72981
    4.3820219 ]
  [ 1.62484097  1.81615591  1.96782994 ...  3.19675279  3.72492218
    4.16049576]
  [ 1.56971455  1.4913969   1.39044571 ...  2.72220159  3.04117537
    3.281

array([[[42.30669192, 44.98092008, 20.28290567,  9.6302772 ,
         21.53555218, 24.81048551, 16.02313601, 12.10834522,
         17.20156998, 27.28302986, 30.39712949, 17.85185931,
         15.10558972,  7.47639372],
        [31.24682374, 31.30482369, 14.41568992,  9.19228371,
         18.69193441, 20.79465449, 15.81637049, 12.09947964,
         13.40892029, 20.89761172, 23.85651583, 12.64878334,
         11.31858938,  5.86271889],
        [24.81272527, 24.4760123 ,  9.86720097, 10.2791315 ,
         17.13138473, 15.96083503, 12.47205154,  9.5523183 ,
         12.03880042, 17.57797848, 20.58796452,  9.49412972,
          8.42232991,  3.8455225 ],
        [10.61820054, 10.94681181,  3.40921455,  0.68424505,
          3.7618156 ,  4.56095911,  3.45737358,  4.02450701,
          5.92919263,  9.11622751,  7.59569085,  5.17493361,
          4.81796859,  1.05660123],
        [ 5.21797195,  4.87480306,  1.8393524 ,  1.00438538,
          4.23235511,  3.35401493,  2.67690526,  2.62104292,
  

In [29]:
# organizando os dados

X = X.reshape(X.shape[0], X.shape[1], X.shape[-1])
print("Formato padronizado dos dados com PSD calculado:", X.shape)

In [30]:
# TAREFA 4

# aplicando a característica de "maior valor"
max_values = np.max(X, axis=-1)
max_values.shape

(160, 1, 9)

In [ ]:
# TAREFA 5

# entendimento dos dados
print(y, y.shape, alvos)

# classificação pelo "voto" do maior valor
i_max = max_values.argmax(axis=-1)
hits = [1 for i in range(len(i_max)) if alvos[i_max[i]] == y[i]]
acc = 100 * sum(hits) / len(y)
print(f'\nPorcentagem de acerto: {acc:.2f}%')